In [1]:
import pandas as pd
import numpy as np
import math

import torch
import torchvision as tv
import torchvision.transforms as transforms

from tqdm import tqdm
from PIL import Image

import cv2
import os
import random

In [49]:
if not os.path.isdir('newimg'):
    os.makedirs('newimg')
WORKING_DIR = '/USER/Taeyun/'

In [50]:
df = pd.read_csv('train.csv')

#validation은 적용하고 싶지 않을 떄
#train = df.loc[:300,:]

In [51]:
covid1 = df[df['COVID']==1]
covid0 = df[df['COVID']==0]

In [52]:
def make_vertical_new(img1,img2):
    upper = img1[:,:np.size(img1,1)//2,:]
    lower = img2[:,np.size(img2,1)//2:,:]
    return np.concatenate([upper,lower],axis=1)

In [53]:
def make_horizon_new(img1,img2):
    lefthalf = img1[:np.size(img1,0)//2,:,:]
    righthalf = img2[np.size(img2,0)//2:,:,:]
    return np.concatenate([lefthalf,righthalf],axis=0)

In [54]:
def make_square_new(img1,img2,img3,img4):
    leftupper = img1[:np.size(img1,0)//2,:np.size(img1,1)//2,:]
    rightupper = img2[:np.size(img2,0)//2,np.size(img2,1)//2:,:]
    leftlower = img3[np.size(img3,0)//2:,:np.size(img3,1)//2,:]
    rightlower = img4[np.size(img4,0)//2:,np.size(img4,1)//2:,:]
    upper = np.concatenate([leftupper,rightupper],axis=1)
    lower = np.concatenate([leftlower,rightlower],axis=1)
    
    return np.concatenate([upper,lower],axis=0)

In [57]:
new_df = {"file_name":[],"COVID":[]}

In [58]:
for i in tqdm(range(646,12646,6)):
    #랜덤으로 이미지 뽑음
    Covid1_1,Covid1_2= random.choices(covid1['file_name'].to_list(),k=2)
    Covid0_1,Covid0_2= random.choices(covid0['file_name'].to_list(),k=2)
    
    # 랜덤으로 뽑은 이미지 가져옴(np.array형태)
    Covid1_img1 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid1_1),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid1_img2 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid1_2),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)

    Covid0_img1 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid0_1),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid0_img2 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid0_2),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    
    #반복하면서 수직,수평으로 분할해서 라벨이 0,1인 이미지 2개생성
    # img를 폴더에 저장
    new_1_ver = make_vertical_new(Covid1_img1,Covid1_img2)
    new_0_ver = make_vertical_new(Covid0_img1,Covid0_img2)
    new_1_hor = make_horizon_new(Covid1_img1,Covid1_img2)
    new_0_hor = make_horizon_new(Covid0_img1,Covid0_img2)
    (Image.fromarray(new_1_ver)).save(f'{WORKING_DIR}newimg/{i}.png','png')
    (Image.fromarray(new_0_ver)).save(f'{WORKING_DIR}newimg/{i+1}.png','png')
    (Image.fromarray(new_1_hor)).save(f'{WORKING_DIR}newimg/{i+2}.png','png')
    (Image.fromarray(new_0_hor)).save(f'{WORKING_DIR}newimg/{i+3}.png','png')
    
    Covid1_1,Covid1_2,Covid1_3,Covid1_4= random.choices(covid1['file_name'].to_list(),k=4)
    Covid0_1,Covid0_2,Covid0_3,Covid0_4 = random.choices(covid0['file_name'].to_list(),k=4)
    
    Covid1_img1 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid1_1),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid1_img2 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid1_2),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid1_img3 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid1_3),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid1_img4 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid1_4),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    
    
    Covid0_img1 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid0_1),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid0_img2 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid0_2),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid0_img3 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid0_3),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid0_img4 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid0_4),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    
    
    new_1_square = make_square_new(Covid1_img1,Covid1_img2,Covid1_img3,Covid1_img4)
    new_0_square = make_square_new(Covid0_img1,Covid0_img2,Covid0_img3,Covid0_img4)
    (Image.fromarray(new_1_square)).save(f"{WORKING_DIR}newimg/{i+4}.png",'png')
    (Image.fromarray(new_0_square)).save(f"{WORKING_DIR}newimg/{i+5}.png",'png')

    # 새로만들어진 img 라벨이 있는 csv작성
    new_df['file_name'].extend([f'{i}.png',f'{i+1}.png',f'{i+2}.png',f'{i+3}.png',f'{i+4}.png',f'{i+5}.png'])
    new_df['COVID'].extend([1,0,1,0,1,0])
        

100% 2000/2000 [07:20<00:00,  4.54it/s]


In [59]:
new_df = pd.DataFrame(new_df)

In [60]:
new_df.to_csv(f'{WORKING_DIR}/new.csv',index=False)